In [ ]:
from pathlib import Path
from utils.get_input import get_input
import numpy as np


inp = Path("./test_input").read_text()
inp = get_input(16)
inp;

In [2]:
import rustworkx as rx

inp_np = np.array([list(i) for i in inp.split("\n")])
inp_np
all_dots = np.argwhere(inp_np == ".")
S = np.argwhere(inp_np == "S")
E = np.argwhere(inp_np == "E")

all_dots_s_e = np.concatenate([all_dots, S, E])

# Build the graph, used in both part1 and 2

num_nodes = len(all_dots_s_e) * 4
all_dots_s_e_map = {tuple(n): i * 4 for i, n in enumerate(all_dots_s_e)}

graph = rx.PyDiGraph()
graph.add_nodes_from(range(num_nodes))

UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

for n, i in all_dots_s_e_map.items():
    y, x = n

    graph.add_edges_from([(i + LEFT, i + UP, 1000)])
    graph.add_edges_from([(i + RIGHT, i + UP, 1000)])
    under = (y + 1, x)
    if under in all_dots_s_e_map:
        graph.add_edges_from([(all_dots_s_e_map[under] + UP, i + UP, 1)])

    graph.add_edges_from([(i + UP, i + RIGHT, 1000)])
    graph.add_edges_from([(i + DOWN, i + RIGHT, 1000)])
    left_of = (y, x - 1)
    if left_of in all_dots_s_e_map:
        graph.add_edges_from([(all_dots_s_e_map[left_of] + RIGHT, i + RIGHT, 1)])

    graph.add_edges_from([(i + RIGHT, i + DOWN, 1000)])
    graph.add_edges_from([(i + LEFT, i + DOWN, 1000)])
    above = (y - 1, x)
    if above in all_dots_s_e_map:
        graph.add_edges_from([(all_dots_s_e_map[above] + DOWN, i + DOWN, 1)])

    graph.add_edges_from([(i + DOWN, i + LEFT, 1000)])
    graph.add_edges_from([(i + UP, i + LEFT, 1000)])
    right_of = (y, x + 1)
    if right_of in all_dots_s_e_map:
        graph.add_edges_from([(all_dots_s_e_map[right_of] + LEFT, i + LEFT, 1)])

s_i = (len(all_dots_s_e) - 2) * 4
e_i = (len(all_dots_s_e) - 1) * 4


ps = rx.dijkstra_shortest_path_lengths(graph, s_i + RIGHT, edge_cost_fn=int)

In [3]:
# from dijkstar import Graph, find_path
def part1():
    c1 = ps[e_i + RIGHT]
    c2 = ps[e_i + DOWN]
    c3 = ps[e_i + LEFT]
    c4 = ps[e_i + UP]

    shortest = int(min(c1, c2, c3, c4))
    return shortest


part1()

94444

In [4]:
%%timeit
part1()

305 ns ± 2.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [5]:
from tqdm import tqdm

# Graph 2, flip all edges
graph2 = rx.PyDiGraph()
graph2.add_nodes_from(range(num_nodes))
edges = []
for fr, to, w in graph.weighted_edge_list():
    edges.append((to, fr, w))
graph2.add_edges_from(edges)
shortest = part1()


def part2():
    sum = 0
    ps = rx.dijkstra_shortest_path_lengths(graph, s_i + RIGHT, edge_cost_fn=int)
    ps2 = rx.dijkstra_shortest_path_lengths(graph2, e_i + RIGHT, edge_cost_fn=int)

    for _, i in list(all_dots_s_e_map.items())[:-2]:
        for d1 in [UP, RIGHT, DOWN, LEFT]:
            c1 = ps[i + d1]
            c2 = ps2[i + d1]
            cost_total = int(c1 + c2)
            if cost_total == shortest:
                sum += 1
                break
    return sum + 2

part2()

502

In [6]:
%%timeit
part2()

19 ms ± 231 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
